<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import os
import numpy as np
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 이미지 저장용 폴더 생성, exist_ok는 해당경로에 같은파일이 이미존재해도 오류를 일으키지 않게함.
os.makedirs("/content/drive/MyDrive/Colab Notebooks/gan/mnist", exist_ok=True)
os.makedirs("/content/drive/MyDrive/Colab Notebooks/gan/gen_imgs", exist_ok=True)

In [ ]:
# 명령줄 인수 파싱 (스크립트로 실행할 경우)
# parser = argparse.ArgumentParser()
# parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
# parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of second order momentum of gradient")
# parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
# parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
# parser.add_argument("--channels", type=int, default=1, help="number of image channels")
# parser.add_argument("--sample_interval", type=int, default=400, help="interval between image samples")
# opt = parser.parse_args()

n_epochs = 200  # 학습 에폭 수
batch_size = 64  # 배치 크기
lr = 0.0002  # 학습률
b1 = 0.5  # adam 최적화기의 베타1 값
b2 = 0.999  # adam 최적화기의 베타2 값
n_cpu = 8  # 배치 생성 시 사용할 CPU 쓰레드 수
latent_dim = 100  # 잠재 공간의 차원 수
img_size = 28  # 이미지의 한 변의 크기
channels = 1  # 이미지의 채널 수
sample_interval = 400  # 이미지 샘플링 간격

img_shape = (channels, img_size, img_size)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        def block(in_dim, out_dim, normalize=True):
            layers = [nn.Linear(in_dim, out_dim)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_dim, momentum=0.8)) # eps는 분모에추가되어 0으로 나누는것을 방지,
                                                             # momentum: 새로운 값과 이전 실행 평균 및 분산에 대한 가중치를 조절하는데 사용되는 값, 기본값은 0.1.
            layers.append(nn.LeakyReLU(0.2, inplace=True)) # 0.2는 LeakyReLU의 음수값 기울기, inplace는 입력텐서를 바로수정해 메모리절약.
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False), # 배치정규화를 사용하지 않는 이유는?
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape) # 이미지 shape 변경.
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512), # np.prod는 곱.
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat) # validity는 타당성.
        return validity

In [ ]:
# 손실함수.
adversarial_loss = nn.BCELoss() # 이진 크로스 엔트로피

# 모델 및 손실함수 초기화.
generator = Generator().to(DEVICE)
discriminator = Discriminator().to(DEVICE)
adversarial_loss = adversarial_loss.to(DEVICE)

In [ ]:
# Configure data loader 데이터로더 구성.
os.makedirs("/content/drive/MyDrive/Colab Notebooks/gan/mnist/data/mnist", exist_ok=True)
dataloader = DataLoader(
    datasets.MNIST(
        "/content/drive/MyDrive/Colab Notebooks/gan/mnist/data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose( # 데이터 변환을 조합. 리사이즈, 텐서변환, 노말라이즈(0.5빼기 0.5나누기. 0에서 1사이의 값을 -1에서 1사이의 값을 변경.)
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size,
    shuffle=True,
)

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

In [ ]:
# Training
for epoch in range(n_epochs): # 입력에폭만큼 반복
    for i, (imgs, _) in enumerate(dataloader): # 인덱스와 이미지

        # Adversarial ground truths
        valid = torch.ones(imgs.size(0), 1, device=DEVICE) # 배치 묶음만큼 행으로하는 1로채워진 텐서
        fake = torch.zeros(imgs.size(0), 1, device=DEVICE)

        # Configure input
        real_imgs = imgs.type(torch.Tensor).to(DEVICE) #이미지의 데이터형을 텐서로 변경

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad() # 그라디언트 초기화

        # Sample noise as generator input
        z = torch.randn(imgs.shape[0], latent_dim, device=DEVICE) # 가우시안분포에서 노이즈 생성 latent_dim은 노이즈벡터의 차원

        # Generate a batch of images
        gen_imgs = generator(z) # 가짜이미지 생성

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid) # valil 실제이미지인척!

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]"
        )

        # 아미지 저장
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], f"/content/drive/MyDrive/Colab Notebooks/gan/gen_imgs/{batches_done}.png", nrow=5, normalize=True)

In [ ]:
#전체 모델 저장하기
#torch.save(model, '/content/drive/MyDrive/Colab Notebooks/results/short_term_model.pt')

# # 나중에 전체 모델 로드하기
# model = torch.load('model_complete.pth')
# model.eval()  # 추론 모드로 설정